In [6]:
from IPython.core.display import HTML

def css_styling():
    styles = open("style/custom.css", "r").read()
    return HTML(styles)
css_styling()

# Javascript in Qualtrics

The file documents a few features that I embedded in Qualtrics during the development of an ego network survey. Qualtrics provide very minimal guidance on how to properly embed Javascript on its platform. The following codes benefit substantially from related questions and solutions provided by users on StackOverflow. Thank you all.

Please be cautious that Qualtrics platform changes from time to time. And therefore the following codes may be outdated at the time you stuble onto this document. Use with care.

Hope it helps and Enjoy!

## Overall tips
* Qualtrics loads everything on a page together. It usually works fine, but sometimes multiple questions may be displayed to participants on one single page. Issues arise when a question needs to retrive a piece of data, but the question that encodes this specific piece of data is on the same page. A solution is to add page breaks between these questions.
* If you want to store a few variables, you need to first add them to "set embedded data" in survey flow before defining them in javascript. 

## Survey overview
* The survey below has four conditions. Participants are randomly selected into one of these four conditions.
* First name generator question: Participants are first given 10 blanks to fill in names. They do not need to fill in all of them. The number of names they generate in this stage is encoded in `nameGen1`. The exact names they generate are encoded in `name1`, `name2`, ... `name10` in sequence.
* Second name generator question: Afterwards, participants are given all the names they have generated in the previous section, and asked to come up with additional names if they can. In this section, they are given a textbox, and instructed to type in additional names in the format of "name1, name2, name3, ...". Javascript code embedded in the question will parse the string by ",", and trim all additional white spaces. The number of names generated in this stage is encded in `nameGen2`. And the exact names participantes generate are encoded in `namep1`, `namep2`, ..., `namep10`. Only the first 10 names are encoded.
* Name interpreter matrix: The first 15 names generated in both stages are displayed as names of rows and columns of the matrix. If a participants generate fewer than 15 names, the additional columns and rows will be automatically marked with a "x". Moreover, participants only needs to fill out the lower triangular matrix. The matrix is assumed to be symmetric and therefore, the upper triangular matrix can be inferred from the lower triangular one.

![Set Embedded Data](image/0.jpeg)


1. Javascript for name generator question. 

    * Question type: text entry -> form 

    * Embed the following code in the first name generator question. 

![Example: name generator question](image/Q1.jpeg)

```javascript
Qualtrics.SurveyEngine.addOnload(function()
{
	/*Place your JavaScript here to run when the page loads*/

});

Qualtrics.SurveyEngine.addOnReady(function()
{
	/*Place your JavaScript here to run when the page is fully displayed*/

});

Qualtrics.SurveyEngine.addOnUnload(function()
{
	/*Place your JavaScript here to run when the page is unloaded*/
	var condition = Qualtrics.SurveyEngine.getEmbeddedData('condition');
	if (condition == 1){
		var count = 0; //count how many names a participant generates
		for (var i = 1; i < 11; i++) {
			var key = 'name' + i;
			var response = $$('.InputText')[i-1].value;
			console.log(key)
			console.log(response)
			if (response.replace(/\s/g, '').length != 0){
				// exclude strings that only contain whitespace (ie. spaces, tabs or line breaks)
				var value = response.trim();
				count ++;
				var tag = 'name' + count;
                // tags are already set up as embedded data in the survey flow.
				Qualtrics.SurveyEngine.setEmbeddedData(tag, value);
			}
		};
		Qualtrics.SurveyEngine.setEmbeddedData('nameGen1', count);
		console.log(count)
	}

});
```

2. Javascript for second name generator question (prompt for additional names). 

    * Question type: text entry.

    * How-tos: Embed the following code in a new question after the second name generator question. Make sure this new question does not appear on the same page as the second name generator question.
    
![Example: additional_name_question1](image/Q2.jpeg)
![Example: additional_name_question2](image/Q3.jpeg)

```javascript
Qualtrics.SurveyEngine.addOnload(function()
{
	/*Place your JavaScript here to run when the page loads*/
	// make sure that nameGen1, nameGen2 and entryLength are set up as number types in Survey Flow.
	
    // change your question ID here
    var entry = "${q://Your_QuestionID_here/ChoiceTextEntryValue}".split(','); 
	console.log(entry)
	var splitText = [];
	var entryLength = 0;
	entry.forEach(function(elem){
		if (elem.replace(/\s/g, '').length != 0){
			// exclude string only contained whitespace (ie. spaces, tabs or line breaks)
			var value = elem.trim();
			splitText.push(value);
			entryLength += 1;
		};
	});
	
	console.log(entryLength);
	console.log(splitText);
		
	var nameL = entryLength < 10 ? entryLength : 10;
	for (var i = 1; i < (nameL+1) ; i++){
		var value = splitText[i-1];
		var tag = 'name_p' + i;
		console.log(value);
		console.log(tag);
		Qualtrics.SurveyEngine.setEmbeddedData(tag, value);
	};
	Qualtrics.SurveyEngine.setEmbeddedData('entryLength', entryLength);
	Qualtrics.SurveyEngine.setEmbeddedData('nameGen2', nameL);
});

Qualtrics.SurveyEngine.addOnReady(function()
{
	/*Place your JavaScript here to run when the page is fully displayed*/

});

Qualtrics.SurveyEngine.addOnUnload(function()
{
	/*Place your JavaScript here to run when the page is unloaded*/	

});
```

3. Javascript for ego tie relations
    * Question type: matrix table -> text entry, short
    * Example: types of ego ties

![Example: ego_tie](image/Q5.jpeg)

```javascript
Qualtrics.SurveyEngine.addOnload(function()
{
	/*Place your JavaScript here to run when the page loads*/
	//http...&Q_JFE=0
	
	var jfe = false;
    if(window.location.pathname.match(/^\/jfe/)) jfe = true;

	//console.log($(this.questionId).down('.Answers').down('th').innerHTML)
	console.log(jfe);	


});

Qualtrics.SurveyEngine.addOnReady(function()
{
	/*Place your JavaScript here to run when the page is fully displayed*/
	
	var nameGen1 = Qualtrics.SurveyEngine.getEmbeddedData('nameGen1');
	var nameGen2 = Qualtrics.SurveyEngine.getEmbeddedData('nameGen2');
	// make sure nameGen1 and nameGen2 should be set as number type in Survey Flow
	for (var i = 0; i < nameGen1; i++){
		var nameID = 'name'+(i+1);
		var name = Qualtrics.SurveyEngine.getEmbeddedData(nameID);
		
		//$$('table.ChoiceStructure thead')[0].select('th')[i].update(name);
		$$('table.ChoiceStructure tbody')[0].select('th')[i].update(name);
	};
	
	for (var i =  0; i <  nameGen2; i++){
		var nameID = 'name_p'+(i+1);
		var name = Qualtrics.SurveyEngine.getEmbeddedData(nameID);
		var seq = Number(i)+Number(nameGen1);
		
		//$$('table.ChoiceStructure thead')[0].select('th')[seq].update(name)
		$$('table.ChoiceStructure tbody')[0].select('th')[seq].update(name)
		
	};

	var nameSum = Number(nameGen1) + Number(nameGen2);
	for (var i = nameSum; i < 15; i++){
		$$('table.ChoiceStructure tbody')[0].select('td')[i].down().value = "x";
	};
	
});

Qualtrics.SurveyEngine.addOnUnload(function()
{
	/*Place your JavaScript here to run when the page is unloaded*/
});
```

4. Javascript for alter-alter relations

    * Question type: matrix table -> text entry, short
    * Example: strength of alter-alter relations

![Example: alter_relations](image/Q4.jpeg)

```javascript
Qualtrics.SurveyEngine.addOnload(function()
{
	/*Place your JavaScript here to run when the page loads*/
	//http...&Q_JFE=0
	
	var jfe = false;
    if(window.location.pathname.match(/^\/jfe/)) jfe = true;

	//console.log($(this.questionId).down('.Answers').down('th').innerHTML)
	console.log(jfe);	


});

Qualtrics.SurveyEngine.addOnReady(function()
{
	/*Place your JavaScript here to run when the page is fully displayed*/
	
	var nameGen1 = Qualtrics.SurveyEngine.getEmbeddedData('nameGen1');
	var nameGen2 = Qualtrics.SurveyEngine.getEmbeddedData('nameGen2');
    var nrow = 15;
	// make sure nameGen1 and nameGen2 are set as number type in Survey Flow
	for (var i = 0; i < nameGen1; i++){
		var nameID = 'name'+(i+1);
		var name = Qualtrics.SurveyEngine.getEmbeddedData(nameID);
		
		$$('table.ChoiceStructure thead')[0].select('th')[i].update(name);
		$$('table.ChoiceStructure tbody')[0].select('th')[i].update(name);
	};
	
	for (var i =  0; i <  nameGen2; i++){
		var nameID = 'name_p'+(i+1);
		var name = Qualtrics.SurveyEngine.getEmbeddedData(nameID);
		var seq = Number(i)+Number(nameGen1);
		
		$$('table.ChoiceStructure thead')[0].select('th')[seq].update(name)
		$$('table.ChoiceStructure tbody')[0].select('th')[seq].update(name)
		
	};
	
	var nameSum = Number(nameGen1) + Number(nameGen2);
	for (var i = nrow*nameSum; i < nrow*nrow; i++){
		$$('table.ChoiceStructure tbody')[0].select('td')[i].down().value = "x";
	};
	
});

Qualtrics.SurveyEngine.addOnUnload(function()
{
	/*Place your JavaScript here to run when the page is unloaded*/

});

```

5. Javascript for surveys

    * Question type: matrix table - Likert, single answer

    * Example: energy question
   
![Energy question](image/4survey.jpeg)

```javascript
Qualtrics.SurveyEngine.addOnload(function()
{
	/*Place your JavaScript here to run when the page loads*/

});

Qualtrics.SurveyEngine.addOnReady(function()
{
	/*Place your JavaScript here to run when the page is fully displayed*/
	var nameGen1 = Qualtrics.SurveyEngine.getEmbeddedData('nameGen');
	console.log(nameGen);
	for (var i = 0; i < nameGen1; i++){
		var nameID = 'name'+(i+1);
		var name = Qualtrics.SurveyEngine.getEmbeddedData(nameID);
		var rowID = "header~QID91~" + (i+1); # adjust QID according to the survey
		console.log(rowID);		
		$(rowID).select('span')[1].update(name);
	};
});

Qualtrics.SurveyEngine.addOnUnload(function()
{
	/*Place your JavaScript here to run when the page is unloaded*/
});

```